***Challenge 2***

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

import numpy as np
from numpy.random import RandomState
import time


# **ResNet50**

In [ ]:
def extract_features(data_loader):
    features = []
    labels = []

    with torch.no_grad():  # We do not need to compute gradients
        for images, targets in data_loader:
            images = images.to(device)
            outputs = resnet(images)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())

    # Flatten the lists
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels


# Load the pretrained ResNet model
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Identity()  # Remove the final fully connected layer

# We need to set the model to evaluation mode and transfer it to the appropriate device
resnet.eval()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
resnet.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 154MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

The below tries  2 random problem instances. In your development you may choose to prototype with 1 problem instances but keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
import time
import torch


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

cifar_data = datasets.CIFAR10(root='.', train=True, transform=transform_train, download=True)
cifar_data_val = datasets.CIFAR10(root='.', train=True, transform=transform_val, download=True)


accs = []
times = []

for seed in range(1, 5):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_loader = DataLoader(Subset(cifar_data, indx_train), batch_size=128, shuffle=True)
  val_loader = DataLoader(Subset(cifar_data_val, indx_val), batch_size=128, shuffle=False)

  # Extract features
  X_train, y_train = extract_features(train_loader)
  X_test, y_test = extract_features(val_loader)

  # Train AdaBoost on the extracted features
  clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50, learning_rate=1.0, random_state=0)
  start_time = time.time()
  clf.fit(X_train, y_train)
  end_time = time.time()

  # Prediction and evaluation
  y_pred = clf.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  accs.append(accuracy * 100)
  times.append(end_time - start_time)
  print(f"Test Accuracy: {accuracy * 100:.2f}%")
  print(classification_report(y_test, y_pred))

# Output the results
accs = np.array(accs)
times = np.array(times)
print('Acc over 5 instances: %.2f +- %.2f' % (accs.mean(), accs.std()))
print(f"Average Time over 5 instances: {times.mean()} +- {times.std()}")

Files already downloaded and verified
Files already downloaded and verified
Test Accuracy: 87.00%
              precision    recall  f1-score   support

           1       0.83      0.94      0.88       200
           9       0.93      0.81      0.86       200

    accuracy                           0.87       400
   macro avg       0.88      0.87      0.87       400
weighted avg       0.88      0.87      0.87       400

Test Accuracy: 84.50%
              precision    recall  f1-score   support

           3       0.84      0.85      0.85       200
           7       0.85      0.83      0.84       200

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.85      0.84      0.84       400

Test Accuracy: 94.50%
              precision    recall  f1-score   support

           1       0.92      0.97      0.95       200
           3       0.97      0.92      0.94       200

    accuracy                           0

# **ResNet18**

In [ ]:
# Load the pretrained ResNet model
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # Remove the final fully connected layer

# We need to set the model to evaluation mode and transfer it to the appropriate device
resnet.eval()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
resnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
import time
import torch


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

cifar_data = datasets.CIFAR10(root='.', train=True, transform=transform_train, download=True)
cifar_data_val = datasets.CIFAR10(root='.', train=True, transform=transform_val, download=True)


accs = []
times = []

for seed in range(1, 5):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_loader = DataLoader(Subset(cifar_data, indx_train), batch_size=128, shuffle=True)
  val_loader = DataLoader(Subset(cifar_data_val, indx_val), batch_size=128, shuffle=False)

  # Extract features
  X_train, y_train = extract_features(train_loader)
  X_test, y_test = extract_features(val_loader)

  # Train AdaBoost on the extracted features
  clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50, learning_rate=1.0, random_state=0)
  start_time = time.time()
  clf.fit(X_train, y_train)
  end_time = time.time()

  # Prediction and evaluation
  y_pred = clf.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  accs.append(accuracy * 100)
  times.append(end_time - start_time)
  print(f"Test Accuracy: {accuracy * 100:.2f}%")
  print(classification_report(y_test, y_pred))

# Output the results
accs = np.array(accs)
times = np.array(times)
print('Acc over 5 instances: %.2f +- %.2f' % (accs.mean(), accs.std()))
print(f"Average Time over 5 instances: {times.mean()} +- {times.std()}")

Files already downloaded and verified
Files already downloaded and verified
Test Accuracy: 83.00%
              precision    recall  f1-score   support

           1       0.79      0.90      0.84       200
           9       0.88      0.76      0.82       200

    accuracy                           0.83       400
   macro avg       0.84      0.83      0.83       400
weighted avg       0.84      0.83      0.83       400

Test Accuracy: 83.25%
              precision    recall  f1-score   support

           3       0.85      0.81      0.83       200
           7       0.82      0.85      0.84       200

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400

Test Accuracy: 97.25%
              precision    recall  f1-score   support

           1       0.99      0.95      0.97       200
           3       0.95      0.99      0.97       200

    accuracy                           0